# Charts for my website

- Author: Kiril from Mindgraph
- Last meaningful update: 01-12-2025

This notebook creates some `altair` charts for [my website](www.mindgraph.dk). The charts are displayed as visuals in this notebook and also exported to `.html` files on GitHub so that they can be directly embedded on the website.

In [1]:
import pandas as pd
import numpy as np
import altair as alt

In [2]:
# Importing data [WIP - to be expanded]
ft_afstemninger = pd.read_parquet("output/ft_afstemninger.parquet")
ft_behandlinger = pd.read_parquet("output/ft_behandlinger.parquet")
results_topics_auto = pd.read_parquet("output/results_topics_auto.parquet")
mapping_topics = pd.read_excel("input/mapping_tabeller.xlsx", sheet_name="Emner")
mapping_countries = pd.read_excel("input/mapping_tabeller.xlsx", sheet_name="Lande")
dst_befolkning = pd.read_parquet("output/dst_befolkning.parquet")
dst_opholdstilladelser = pd.read_parquet("output/dst_opholdstilladelser.parquet")
dst_statsborgerskaber = pd.read_parquet("output/dst_statsborgerskab.parquet")

In [3]:
# Creating a custom color palette for charts
# Using an extended version of the "Classy" palette from:
# https://mycolor.space/?hex=%231EA2B5&sub=1

my_custom_palette = [
    "#1ea2b5", "#324b4f", "#95b0b5", "#9f8ac3", "#6b588d",  # original
    "#009170",
    "#005545",
    "#0d221c",
    "#005c87",
    "#e6f4f1",
    "#eee8a9",
    "#a4943e",
    "#d471bb",
    "#27c0bf",
    "#55ddbe",
    "#8df8b7",
    "#de6f9f",
    "#70a153",
    "#885534"
]

## Figure 1

INSERT CHART WITH % OF VOTES BY FOR/AGAINST/ETC ACROSS TIME - STACKED TO 100%

In [4]:
# Aggregating data for chart
id_cols = ["År", "Stemme"]
cols_keep = id_cols + ["AndelStemmer"]
chart_data = ft_afstemninger[ft_afstemninger["Stemme"] != "Fraværende"].copy()
chart_data["AntalStemmer"] = chart_data.groupby(id_cols)["Sæson"].transform("count")
chart_data["AlleStemmer"] = chart_data.groupby("År")["Sæson"].transform("count")
chart_data["AndelStemmer"] = chart_data["AntalStemmer"] / chart_data["AlleStemmer"]
chart_data = chart_data.drop_duplicates(id_cols)

In [5]:
# Renaming columns for chart
col_names = {"AndelStemmer": "% af alle stemmer"}
chart_data_dk = chart_data.rename(columns=col_names)

# Specifying custom sort order for the color variable
custom_val_order = ["For", "Imod", "Hverken for eller imod"]
vote_order_mapping = {v: i for i, v in enumerate(custom_val_order)}
chart_data_dk["CustomOrder"] = chart_data_dk["Stemme"].map(vote_order_mapping)

# Creating a chart in Danish
chart = (
    alt.Chart(chart_data_dk, width="container")
    .mark_area()
    .encode(
        x="År:O",
        y=alt.Y("% af alle stemmer:Q", axis=alt.Axis(format=".1%")),
        color=alt.Color(
            "Stemme:N",
            scale=alt.Scale(domain=custom_val_order, range=my_custom_palette),
        ),
        order=alt.Order("CustomOrder:Q"),
        tooltip=["År:O", "Stemme:N", alt.Tooltip("% af alle stemmer:Q", format=".1%")],
    )
)
chart.save(".charts/figure_01_dk.html")
chart

alt.Chart(...)

In [6]:
# Renaming columns for chart
col_names = {"AndelStemmer": "% of all votes", "År": "Year", "Stemme": "Vote"}
chart_data_en = chart_data.rename(columns=col_names)

# Replacing certain values
val_mapping = {
    "For": "For",
    "Imod": "Against",
    "Hverken for eller imod": "Neither for nor against",
}
chart_data_en["Vote"] = chart_data_en["Vote"].replace(val_mapping)

# Specifying custom sort order for the color variable
custom_val_order = ["For", "Against", "Neither for nor against"]
vote_order_mapping = {v: i for i, v in enumerate(custom_val_order)}
chart_data_en["CustomOrder"] = chart_data_en["Vote"].map(vote_order_mapping)

# Creating a chart in English
chart = (
    alt.Chart(chart_data_en, width="container")
    .mark_area()
    .encode(
        x="Year:O",
        y=alt.Y("% of all votes:Q", axis=alt.Axis(format=".1%")),
        color=alt.Color(
            "Vote:N",
            scale=alt.Scale(domain=custom_val_order, range=my_custom_palette),
        ),
        order=alt.Order("CustomOrder:Q"),
        tooltip=["Year:O", "Vote:N", alt.Tooltip("% of all votes:Q", format=".1%")],
    )
)
chart.save(".charts/figure_01_en.html")
chart

alt.Chart(...)

## Figure 2

INSERT CHART WITH TOTAL % OF VOTES FOR/AGAINST/ETC BY PARTY

In [7]:
# Aggregating data for chart
id_cols = ["PartiGruppe", "Stemme"]
sort_cols = ["PartiGruppe", "Stemme"]
cols_keep = id_cols + ["AndelStemmer"]
chart_data = ft_afstemninger[ft_afstemninger["Stemme"] != "Fraværende"].copy()
chart_data["AntalStemmer"] = chart_data.groupby(id_cols)["Sæson"].transform("count")
chart_data["AlleStemmer"] = chart_data.groupby("PartiGruppe")["Sæson"].transform(
    "count"
)
chart_data["AndelStemmer"] = chart_data["AntalStemmer"] / chart_data["AlleStemmer"]
chart_data = chart_data.drop_duplicates(id_cols)
chart_data = chart_data.sort_values(by=sort_cols)

In [8]:
# Renaming columns for chart
col_names = {"AndelStemmer": "% af alle stemmer", "PartiGruppe": "Partigruppe"}
chart_data_dk = chart_data.rename(columns=col_names)

# Specifying custom sort order for the color variable
custom_val_order = ["For", "Imod", "Hverken for eller imod"]
vote_order_mapping = {v: i for i, v in enumerate(custom_val_order)}
chart_data_dk["CustomOrder"] = chart_data_dk["Stemme"].map(vote_order_mapping)

# Creating a chart in Danish
chart = (
    alt.Chart(chart_data_dk, width="container")
    .mark_bar()
    .encode(
        x=alt.X("% af alle stemmer:Q", axis=alt.Axis(format=".1%")),
        y="Partigruppe",
        color=alt.Color(
            "Stemme:N",
            scale=alt.Scale(domain=custom_val_order, range=my_custom_palette),
        ),
        order=alt.Order("CustomOrder:Q"),
        tooltip=[
            "Partigruppe:O",
            "Stemme:N",
            alt.Tooltip("% af alle stemmer:Q", format=".1%"),
        ],
    )
)
chart.save(".charts/figure_02_dk.html")
chart

alt.Chart(...)

In [9]:
# Renaming columns for chart
col_names = {"AndelStemmer": "% of all votes", "PartiGruppe": "Party", "Stemme": "Vote"}
chart_data_en = chart_data.rename(columns=col_names)

# Replacing certain values
val_mapping = {
    "For": "For",
    "Imod": "Against",
    "Hverken for eller imod": "Neither for nor against",
}
chart_data_en["Vote"] = chart_data_en["Vote"].replace(val_mapping)

# Specifying custom sort order for the color variable
custom_val_order = ["For", "Against", "Neither for nor against"]
vote_order_mapping = {v: i for i, v in enumerate(custom_val_order)}
chart_data_en["CustomOrder"] = chart_data_en["Vote"].map(vote_order_mapping)

# Creating a chart in English
chart = (
    alt.Chart(chart_data_en, width="container")
    .mark_bar()
    .encode(
        x=alt.X("% of all votes:Q", axis=alt.Axis(format=".1%")),
        y="Party",
        color=alt.Color(
            "Vote:N",
            scale=alt.Scale(domain=custom_val_order, range=my_custom_palette),
        ),
        order=alt.Order("CustomOrder:Q"),
        tooltip=[
            "Party:O",
            "Vote:N",
            alt.Tooltip("% of all votes:Q", format=".1%"),
        ],
    )
)
chart.save(".charts/figure_02_en.html")
chart

alt.Chart(...)

## Figure 3

<INSERT CHART WITH N OF SPEECHES BY YEAR, USE 2025-1 (-F) instead of just 2025>

In [10]:
# Aggregating data for chart
chart_data = ft_behandlinger.copy()
chart_data["AntalUdtalelser"] = chart_data.groupby("År")["Udtalelse"].transform("count")
chart_data = chart_data.drop_duplicates("År")
chart_data = chart_data.sort_values("År")
chart_data = chart_data[["År", "AntalUdtalelser"]]

In [11]:
col_names = {"AntalUdtalelser":"Antal udtalelser"}
chart_data_dk = chart_data.rename(columns=col_names)

# Creating a chart in Danish
chart = alt.Chart(chart_data_dk, width="container").mark_line(color=my_custom_palette[0]).encode(
    x='År:O',
    y='Antal udtalelser:Q',
    tooltip=["År:O", "Antal udtalelser:Q"]
)
chart.save(".charts/figure_03_dk.html")
chart

alt.Chart(...)

In [12]:
# Renaming columns for chart
col_names = {"År":"Year", "AntalUdtalelser":"Number of speeches"}
chart_data_en = chart_data.rename(columns=col_names)

# Creating a chart in English
chart = alt.Chart(chart_data_en, width="container").mark_line(color=my_custom_palette[0]).encode(
    x='Year:O',
    y='Number of speeches:Q',
    tooltip=["Year:O", "Number of speeches:Q"]
)
chart.save(".charts/figure_03_en.html")
chart

alt.Chart(...)

## Figure 4

<PIE CHART with the total % either in time or N of words in the debates for all parties>

In [13]:
# Aggregating data for chart
chart_data = ft_behandlinger.copy()
chart_data["LængdeParti"] = chart_data.groupby("PartiGruppe")["UdtalelseLængdeMinutter"].transform("sum")
chart_data["LængdeSamlet"] = chart_data["UdtalelseLængdeMinutter"].sum()
chart_data = chart_data.drop_duplicates("PartiGruppe")
chart_data["AndelParti"] = chart_data["LængdeParti"] / chart_data["LængdeSamlet"]
chart_data = chart_data[["PartiGruppe", "AndelParti"]]
chart_data = chart_data.sort_values("AndelParti", ascending=False)

In [14]:
# Renaming columns for chart
col_names = {"PartiGruppe":"Partigruppe", "AndelParti":"Andel i debatten"}
chart_data_dk = chart_data.rename(columns=col_names)

# Defining custom sort order
sort_order = chart_data_dk["Partigruppe"].tolist()

# Creating a chart in Danish
chart = alt.Chart(chart_data_dk, width="container").mark_arc(innerRadius=70).encode(
    theta=alt.Theta("Andel i debatten:Q"),
    color=alt.Color(
        "Partigruppe:N",
        sort=sort_order,
        scale=alt.Scale(range=my_custom_palette)
    ),
    tooltip=[
        alt.Tooltip("Partigruppe:N"),
        alt.Tooltip("Andel i debatten:Q", format=".1%")
    ]
)
chart.save(".charts/figure_04_dk.html")
chart

alt.Chart(...)

In [15]:
# Renaming columns for chart
col_names = {"PartiGruppe":"Party", "AndelParti":"Share in the debate"}
chart_data_en = chart_data.rename(columns=col_names)

# Defining custom sort order
sort_order = chart_data_en["Party"].tolist()

# Creating a chart in English
chart = alt.Chart(chart_data_en, width="container").mark_arc(innerRadius=70).encode(
    theta=alt.Theta("Share in the debate:Q"),
    color=alt.Color(
            "Party:N",
            sort=sort_order,
            scale=alt.Scale(range=my_custom_palette)
        ),
    tooltip=[
        alt.Tooltip("Party:N"),
        alt.Tooltip("Share in the debate:Q", format=".1%")
    ]
)
chart.save(".charts/figure_04_en.html")
chart

alt.Chart(...)

## Figure 5

<LINE CHART with years and parties in the legend where Y = % in the debate divided by % in FT>

In [16]:
# Aggregating data for chart
party_shares = ft_afstemninger.copy()
id_cols = ["År", "PartiGruppe"]
party_shares["AntalMF_Parti"] = party_shares.groupby(id_cols)["Navn"].transform("count")
party_shares["AntalMF_Alle"] = party_shares.groupby("År")["Navn"].transform("count")
party_shares["AndelIFT"] = party_shares["AntalMF_Parti"]/party_shares["AntalMF_Alle"]
party_shares = party_shares.drop_duplicates(id_cols)
party_shares = party_shares[id_cols + ["AndelIFT"]]

debate_shares = ft_behandlinger.copy()
debate_shares["AntalMinParti"] = debate_shares.groupby(id_cols)["UdtalelseLængdeMinutter"].transform("sum")
debate_shares["AntalMinAlle"] = debate_shares.groupby("År")["UdtalelseLængdeMinutter"].transform("sum")
debate_shares["AndelIDebat"] = debate_shares["AntalMinParti"]/debate_shares["AntalMinAlle"]
debate_shares = debate_shares.drop_duplicates(id_cols)
debate_shares = debate_shares[id_cols + ["AndelIDebat"]]

shares = pd.merge(party_shares, debate_shares, how="left", on=id_cols)
shares["AndelIDebat"] = shares["AndelIDebat"].fillna(0)
shares["PartiRatio"] = shares["AndelIDebat"]/shares["AndelIFT"]

In [17]:
col_names = {"PartiRatio":"Repræsentation", "PartiGruppe":"Partigruppe"}
chart_data_dk = shares.rename(columns=col_names)

# Creating a chart in Danish
chart = alt.Chart(chart_data_dk, width="container").mark_line(color=my_custom_palette[0]).encode(
    x='År:O',
    y='Repræsentation:Q',
    color=alt.Color(
                'Partigruppe:N',
                scale=alt.Scale(range=my_custom_palette)
            ),
    tooltip=["År:O", 'Partigruppe:N', alt.Tooltip("Repræsentation:Q", format=".1f")]
)
chart.save(".charts/figure_05_dk.html")
chart

alt.Chart(...)

In [18]:
col_names = {"År":"Year", "PartiRatio":"Representation", "PartiGruppe":"Party"}
chart_data_en = shares.rename(columns=col_names)

# Creating a chart in Danish
chart = alt.Chart(chart_data_en, width="container").mark_line(color=my_custom_palette[0]).encode(
    x='Year:O',
    y='Representation:Q',
    color=alt.Color(
                'Party:N',
                scale=alt.Scale(range=my_custom_palette)
            ),
    tooltip=["Year:O", 'Party:N', alt.Tooltip("Representation:Q", format=".1f")]
)
chart.save(".charts/figure_05_en.html")
chart

alt.Chart(...)

## Figure 6

<PIE CHART with the total % of speeches divided by overall topic>

In [19]:
# Aggregating data for chart
chart_data = pd.merge(results_topics_auto, mapping_topics, how="left", on="EmneNr")
chart_data["AntalUdtalelser"] = chart_data.groupby("EmneGruppe")["UdtalelseId"].transform("count")
chart_data = chart_data.drop_duplicates("EmneGruppe")
chart_data = chart_data.sort_values("AntalUdtalelser", ascending=False)
chart_data["AndelUdtalelser"] = chart_data["AntalUdtalelser"]/chart_data["AntalUdtalelser"].sum()
chart_data = chart_data[["EmneGruppe", "AndelUdtalelser"]]

In [20]:
# Renaming columns for chart
col_names = {"EmneGruppe":"Emnegruppe", "AndelUdtalelser":"Andel af alle udtalelser"}
chart_data_dk = chart_data.rename(columns=col_names)

# Defining custom sort order
sort_order = chart_data_dk["Emnegruppe"].tolist()

# Making a chart in Danish
chart = (
    alt.Chart(chart_data_dk, width="container")
    .mark_arc(innerRadius=70)
    .encode(
        theta=alt.Theta("Andel af alle udtalelser:Q"),
        color=alt.Color(
            "Emnegruppe:N",
            sort=sort_order,
            scale=alt.Scale(range=my_custom_palette)
        ),
        tooltip=[
            alt.Tooltip("Emnegruppe:N"),
            alt.Tooltip("Andel af alle udtalelser:Q", format=".1%"),
        ],
    )
)
chart.save(".charts/figure_06_dk.html")
chart

alt.Chart(...)

In [21]:
# Renaming columns for chart
col_names = {"EmneGruppe":"Topic group", "AndelUdtalelser":"Share of all speeches"}
chart_data_en = chart_data.rename(columns=col_names)

# Replacing values from the mapping table
val_replace = {'Der er behov for tydeligere regler og retssikkerhed i ansøgningsprocessen':"There is a need for clearer rules and legal certainty in the process",
 'Statsborgerskab fremmer integration og belønner ansøgernes bidrag til samfundet':"Citizenship favours integration and rewards applicants' contribution to society",
 'Der er problemer med kulturelle forskelle ifm. ikke-vestlig indvandring':"There are issues with cultural differences related to non-Western migrants",
 'Ren procedure snak/politiker peger fingre på hinanden':"Procedural talk/politicians pointing fingers at each other",
 'Diskussion af politiske tiltag fra regeringen/partiernes side ifm. indfødsretsprocessen':"Discussion of political measures related to the application process",
 'Der er bekymringer for utilstrækkelig integration og kriminalitet blandt dem, som tildeles indfødsret':"There are concerns reg. insufficient integration and crime among those obtaining citizenship",
 'Reglerne for opnåelsen af dansk statsborgerskab bør strammes':"The requirements for getting Danish citizenship should be made stricter"}
chart_data_en["Topic group"] = chart_data_en["Topic group"].replace(val_replace)

# Defining custom sort order
sort_order = chart_data_en["Topic group"].tolist()
# Making a chart in Danish
chart = (
    alt.Chart(chart_data_en, width="container")
    .mark_arc(innerRadius=70)
    .encode(
        theta=alt.Theta("Share of all speeches:Q"),
        color=alt.Color(
            "Topic group:N",
            sort=sort_order,
            scale=alt.Scale(range=my_custom_palette)
        ),
        tooltip=[
            alt.Tooltip("Topic group:N"),
            alt.Tooltip("Share of all speeches:Q", format=".1%"),
        ],
    )
)
chart.save(".charts/figure_06_en.html")
chart

alt.Chart(...)

## Figure 7

<PIE CHART with the total % of time of all speeches divided by overall topic>

In [22]:
# Aggregating data for chart
chart_data = pd.merge(results_topics_auto, mapping_topics, how="left", on="EmneNr")
chart_data = pd.merge(ft_behandlinger, chart_data, how="right", on="UdtalelseId")
chart_data["AntalMinutter"] = chart_data.groupby("EmneGruppe")["UdtalelseLængdeMinutter"].transform("sum")
chart_data = chart_data.drop_duplicates("EmneGruppe")
chart_data = chart_data.sort_values("AntalMinutter", ascending=False)
chart_data = chart_data[chart_data["EmneGruppe"].notna()]
chart_data = chart_data.drop_duplicates("EmneGruppe")
chart_data["AndelMinutter"] = chart_data["AntalMinutter"]/chart_data["AntalMinutter"].sum()
chart_data = chart_data[["EmneGruppe", "AntalMinutter", "AndelMinutter"]]

In [23]:
# Renaming columns for chart
col_names = {"EmneGruppe":"Emnegruppe", "AndelMinutter":"Andel af alle udtalelser"}
chart_data_dk = chart_data.rename(columns=col_names)

# Defining custom sort order
sort_order = chart_data_dk["Emnegruppe"].tolist()

# Making a chart in Danish
chart = (
    alt.Chart(chart_data_dk, width="container")
    .mark_arc(innerRadius=70)
    .encode(
        theta=alt.Theta("Andel af alle udtalelser:Q"),
        color=alt.Color(
            "Emnegruppe:N",
            sort=sort_order,
            scale=alt.Scale(range=my_custom_palette)
        ),
        tooltip=[
            alt.Tooltip("Emnegruppe:N"),
            alt.Tooltip("Andel af alle udtalelser:Q", format=".1%"),
        ],
    )
)
chart.save(".charts/figure_07_dk.html")
chart

alt.Chart(...)

In [24]:
# Renaming columns for chart
col_names = {"EmneGruppe":"Topic group", "AndelMinutter":"Share of all speeches"}
chart_data_en = chart_data.rename(columns=col_names)

# Replacing values from the mapping table
val_replace = {'Der er behov for tydeligere regler og retssikkerhed i ansøgningsprocessen':"There is a need for clearer rules and legal certainty in the process",
 'Statsborgerskab fremmer integration og belønner ansøgernes bidrag til samfundet':"Citizenship favours integration and rewards applicants' contribution to society",
 'Der er problemer med kulturelle forskelle ifm. ikke-vestlig indvandring':"There are issues with cultural differences related to non-Western migrants",
 'Ren procedure snak/politiker peger fingre på hinanden':"Procedural talk/politicians pointing fingers at each other",
 'Diskussion af politiske tiltag fra regeringen/partiernes side ifm. indfødsretsprocessen':"Discussion of political measures related to the application process",
 'Der er bekymringer for utilstrækkelig integration og kriminalitet blandt dem, som tildeles indfødsret':"There are concerns reg. insufficient integration and crime among those obtaining citizenship",
 'Reglerne for opnåelsen af dansk statsborgerskab bør strammes':"The requirements for getting Danish citizenship should be made stricter"}
chart_data_en["Topic group"] = chart_data_en["Topic group"].replace(val_replace)

# Defining custom sort order
sort_order = chart_data_en["Topic group"].tolist()
# Making a chart in Danish
chart = (
    alt.Chart(chart_data_en, width="container")
    .mark_arc(innerRadius=70)
    .encode(
        theta=alt.Theta("Share of all speeches:Q"),
        color=alt.Color(
            "Topic group:N",
            sort=sort_order,
            scale=alt.Scale(range=my_custom_palette)
        ),
        tooltip=[
            alt.Tooltip("Topic group:N"),
            alt.Tooltip("Share of all speeches:Q", format=".1%"),
        ],
    )
)
chart.save(".charts/figure_07_en.html")
chart

alt.Chart(...)

# Figure 8

Figur 8. Indekserede ændringer i Danmarks samlede befolkningstal samt antallet af uddelte opholdstilladelser og statsborgerskaber (2004=100)
<Pretty self-explanatory based on the Figure’s title, include 4 things: total population, % of the population without Danish citizenship, total residence permits & total naturalizations>

**WIP as of 01-12-2025**: Make sure that the order of the categories is the same in both languages. This applies to Figure 8 and also the other charts.

In [25]:
# Aggregating population data with index 2004=100
tmp_befolkning = dst_befolkning.copy()
tmp_befolkning["Befolkning"] = tmp_befolkning.groupby("År")["AntalMennesker"].transform("sum")
tmp_befolkning = tmp_befolkning.drop_duplicates("År")
tmp_befolkning["Base"] = np.where(tmp_befolkning["År"] == "2004", tmp_befolkning["Befolkning"], np.nan)
tmp_befolkning["Base"] = tmp_befolkning["Base"].bfill().ffill()
tmp_befolkning["BefolkningIndeks"] = 100 * (tmp_befolkning["Befolkning"]/tmp_befolkning["Base"])
tmp_befolkning = tmp_befolkning[["År", "BefolkningIndeks"]]

In [26]:
# Aggregating data on non-Danish citizens with index 2004=100
tmp_indvandrer = dst_befolkning[dst_befolkning["Statsborgerskab"] != "Dansk"].copy()
tmp_indvandrer["Indvandrere"] = tmp_indvandrer.groupby("År")["AntalMennesker"].transform("sum")
tmp_indvandrer = tmp_indvandrer.drop_duplicates("År")
tmp_indvandrer["Base"] = np.where(tmp_indvandrer["År"] == "2004", tmp_indvandrer["Indvandrere"], np.nan)
tmp_indvandrer["Base"] = tmp_indvandrer["Base"].bfill().ffill()
tmp_indvandrer["IndvandrerIndeks"] = 100 * (tmp_indvandrer["Indvandrere"]/tmp_indvandrer["Base"])
tmp_indvandrer = tmp_indvandrer[["År", "IndvandrerIndeks"]]

In [27]:
# Aggregating data on residence permits with index 2004=100
tmp_ophold = dst_opholdstilladelser.copy()
tmp_ophold["Opholdstilladelser"] = tmp_ophold.groupby("År")["AntalMennesker"].transform("sum")
tmp_ophold = tmp_ophold.drop_duplicates("År")
tmp_ophold["Base"] = np.where(tmp_ophold["År"] == "2004", tmp_ophold["Opholdstilladelser"], np.nan)
tmp_ophold["Base"] = tmp_ophold["Base"].bfill().ffill()
tmp_ophold["OpholdstilladelserIndeks"] = 100 * (tmp_ophold["Opholdstilladelser"]/tmp_ophold["Base"])
tmp_ophold = tmp_ophold[["År", "OpholdstilladelserIndeks"]]

In [28]:
# Aggregating data on acquired citizenships with index 2004=100
tmp_statsborgerskab = dst_statsborgerskaber.copy()
tmp_statsborgerskab["Statsborgerskaber"] = tmp_statsborgerskab.groupby("År")["AntalMennesker"].transform("sum")
tmp_statsborgerskab = tmp_statsborgerskab.drop_duplicates("År")
tmp_statsborgerskab["Base"] = np.where(tmp_statsborgerskab["År"] == "2004", tmp_statsborgerskab["Statsborgerskaber"], np.nan)
tmp_statsborgerskab["Base"] = tmp_statsborgerskab["Base"].bfill().ffill()
tmp_statsborgerskab["StatsborgerskaberIndeks"] = 100 * (tmp_statsborgerskab["Statsborgerskaber"]/tmp_statsborgerskab["Base"])
tmp_statsborgerskab = tmp_statsborgerskab[["År", "StatsborgerskaberIndeks"]]

In [29]:
# Combining the data in a single df for use on chart
chart_data = pd.merge(tmp_befolkning, tmp_indvandrer, how="left", on="År")
chart_data = pd.merge(chart_data, tmp_ophold, how="left", on="År")
chart_data = pd.merge(chart_data, tmp_statsborgerskab, how="left", on="År")
chart_data["År"] = pd.to_numeric(chart_data["År"])
chart_data = chart_data[chart_data["År"] >= 2004]
# chart_data = chart_data.dropna()

value_vars = ["BefolkningIndeks", "IndvandrerIndeks", "OpholdstilladelserIndeks", "StatsborgerskaberIndeks",]
chart_data = pd.melt(chart_data, id_vars=["År"], value_vars = value_vars, value_name="Indeks (2004=100)", var_name="Indikator")

val_replace = {"BefolkningIndeks":"Befolkning", "IndvandrerIndeks":"Antal indvandrere", "OpholdstilladelserIndeks":"Opholdstilladelser", "StatsborgerskaberIndeks":"Statsborgerskaber"}
chart_data["Indikator"] = chart_data["Indikator"].replace(val_replace)

In [30]:
# Creating a chart in Danish
chart_data_dk = chart_data
chart = alt.Chart(chart_data_dk, width="container").mark_line(color=my_custom_palette[0]).encode(
    x='År:O',
    y='Indeks (2004=100):Q',
    color=alt.Color(
                'Indikator:N',
                scale=alt.Scale(range=my_custom_palette)
            ),
    tooltip=["År:O", 'Indikator:N', alt.Tooltip("Indeks (2004=100):Q", format=".1f")]
)
chart.save(".charts/figure_08_dk.html")
chart

alt.Chart(...)

In [31]:
# Renaming columns for the chart
col_names = {"År":"Year", "Indikator":"Indicator", "Indeks (2004=100)":"Index (2004=100)"}
chart_data_en = chart_data.rename(columns=col_names)

# Replacing values
val_replace = {"Befolkning":"Population", "Antal indvandrere":"Number of foreigners", "Opholdstilladelser":"Residence permits", "Statsborgerskaber":"Citizenships"}
chart_data_en["Indicator"] = chart_data_en["Indicator"].replace(val_replace)

# Creating a chart in English
chart = alt.Chart(chart_data_en, width="container").mark_line(color=my_custom_palette[0]).encode(
    x='Year:O',
    y='Index (2004=100):Q',
    color=alt.Color(
                'Indicator:N',
                scale=alt.Scale(range=my_custom_palette)
            ),
    tooltip=["Year:O", 'Indicator:N', alt.Tooltip("Index (2004=100):Q", format=".1f")]
)
chart.save(".charts/figure_08_en.html")
chart

alt.Chart(...)

## Figure 9

Figur 9. Samlet antal opholdstilladelser mellem 2004-2024 fordelt pr. herkomstregion

In [32]:
# Preparing data for chart
col_names = {"Land":"Herkomstland", "Region":"Herkomstregion"}
chart_data = dst_opholdstilladelser.copy()
chart_data["År"] = pd.to_numeric(chart_data["År"])
chart_data = chart_data[chart_data["År"] >= 2004]
# chart_data = chart_data[chart_data["År"] <= 2004]
tmp_mapping = mapping_countries.rename(columns=col_names)
chart_data = pd.merge(chart_data, tmp_mapping, how="left", on="Herkomstland")
chart_data["Opholdstilladelser"] = chart_data.groupby("Herkomstregion")["AntalMennesker"].transform("sum")
chart_data = chart_data.drop_duplicates("Herkomstregion")
chart_data = chart_data.sort_values("Opholdstilladelser", ascending=False)
chart_data = chart_data[["Herkomstregion", "Opholdstilladelser"]]
chart_data["Andel"] = chart_data["Opholdstilladelser"]/chart_data["Opholdstilladelser"].sum()

In [33]:
# Renaming columns for chart
col_names = {"Herkomstregion":"Herkomstregion", "Andel":"Andel af alle opholdstilladelser"}
chart_data_dk = chart_data.rename(columns=col_names)

# Defining custom sort order
sort_order = chart_data_dk["Herkomstregion"].tolist()

# Making a chart in Danish
chart = (
    alt.Chart(chart_data_dk, width="container")
    .mark_arc(innerRadius=70)
    .encode(
        theta=alt.Theta("Andel af alle opholdstilladelser:Q"),
        color=alt.Color(
            "Herkomstregion:N",
            sort=sort_order,
            scale=alt.Scale(range=my_custom_palette)
        ),
        tooltip=[
            alt.Tooltip("Herkomstregion:N"),
            alt.Tooltip("Andel af alle opholdstilladelser:Q", format=".1%"),
        ],
    )
)
chart.save(".charts/figure_09_dk.html")
chart

alt.Chart(...)

In [34]:
# Renaming columns for chart
col_names = {"Herkomstregion":"Region of origin", "Andel":"Share of residence permits"}
chart_data_en = chart_data.rename(columns=col_names)

# Replacing values
val_replace = {'Andre vestlige lande':"Other Western countries",
 'Andre ikke-vestlige lande':"Other Non-Western countries",
 'MENAPT-lande':"MENAPT countries",
 'Statsløs':"Stateless",
 'Nordiske lande':"Nordic countries",
 'Uoplyst':"Unspecified"}
chart_data_en["Region of origin"] = chart_data_en["Region of origin"].replace(val_replace)

# Defining custom sort order
sort_order = chart_data_en["Region of origin"].tolist()

# Making a chart in Danish
chart = (
    alt.Chart(chart_data_en, width="container")
    .mark_arc(innerRadius=70)
    .encode(
        theta=alt.Theta("Share of residence permits:Q"),
        color=alt.Color(
            "Region of origin:N",
            sort=sort_order,
            scale=alt.Scale(range=my_custom_palette)
        ),
        tooltip=[
            alt.Tooltip("Region of origin:N"),
            alt.Tooltip("Share of residence permits:Q", format=".1%"),
        ],
    )
)
chart.save(".charts/figure_09_en.html")
chart

alt.Chart(...)

## Figure 10

Figur 10. Uddelte statsborgerskaber per. herkomstregion over tid
<Stacked % chart over time, just as in Power BI>

In [35]:
# Aggregating data for chart
# Preparing data for chart
col_names = {"Land":"Herkomstland", "Region":"Herkomstregion"}
id_cols = ["År", "Herkomstregion"]
chart_data = dst_statsborgerskaber.copy()
tmp_mapping = mapping_countries.rename(columns=col_names)
chart_data["År"] = pd.to_numeric(chart_data["År"])
chart_data = chart_data[chart_data["År"] >= 2004]
# chart_data = chart_data[chart_data["År"] <= 2004]
chart_data = pd.merge(chart_data, tmp_mapping, how="left", on="Herkomstland")
chart_data["Statsborgerskaber"] = chart_data.groupby(id_cols)["AntalMennesker"].transform("sum")
chart_data["ÅrligeStb"] = chart_data.groupby("År")["AntalMennesker"].transform("sum")
chart_data = chart_data.drop_duplicates(id_cols)
chart_data["Andel"] = chart_data["Statsborgerskaber"]/chart_data["ÅrligeStb"]
chart_data = chart_data.sort_values(["År", "Statsborgerskaber"], ascending=False)

In [36]:
# Renaming columns for chart
col_names = {"Andel": "Andel statsborgerskaber"}
chart_data_dk = chart_data.rename(columns=col_names)

# Specifying custom sort order for the color variable
custom_val_order = ["Andre vestlige lande", "Andre ikke-vestlige lande", "MENAPT-lande", "Nordiske lande", "Statløs", "Uoplyst"]
vote_order_mapping = {v: i for i, v in enumerate(custom_val_order)}
chart_data_dk["CustomOrder"] = chart_data_dk["Herkomstregion"].map(vote_order_mapping)

# Creating a chart in Danish
chart = (
    alt.Chart(chart_data_dk, width="container")
    .mark_area()
    .encode(
        x="År:O",
        y=alt.Y("Andel statsborgerskaber:Q", axis=alt.Axis(format=".1%")),
        color=alt.Color(
            "Herkomstregion:N",
            scale=alt.Scale(domain=custom_val_order, range=my_custom_palette),
        ),
        order=alt.Order("CustomOrder:Q"),
        tooltip=["År:O", "Herkomstregion:N", alt.Tooltip("Andel statsborgerskaber:Q", format=".1%")],
    )
)
chart.save(".charts/figure_10_dk.html")
chart

alt.Chart(...)

In [37]:
# Renaming columns for chart
col_names = {"Herkomstregion":"Region of origin", "År":"Year", "Andel": "Share of citizenships"}
chart_data_en = chart_data.rename(columns=col_names)

# Replacing values
val_replace = {'Andre vestlige lande':"Other Western countries",
 'Andre ikke-vestlige lande':"Other Non-Western countries",
 'MENAPT-lande':"MENAPT countries",
 'Statsløs':"Stateless",
 'Nordiske lande':"Nordic countries",
 'Uoplyst':"Unspecified"}
chart_data_en["Region of origin"] = chart_data_en["Region of origin"].replace(val_replace)

# Specifying custom sort order for the color variable
custom_val_order = ["Other Western countries", "Other Non-Western countries", "MENAPT countries", "Nordic countries", "Stateless", "Unspecified"]
vote_order_mapping = {v: i for i, v in enumerate(custom_val_order)}
chart_data_en["CustomOrder"] = chart_data_en["Region of origin"].map(vote_order_mapping)

# Creating a chart in English
chart = (
    alt.Chart(chart_data_en, width="container")
    .mark_area()
    .encode(
        x="Year:O",
        y=alt.Y("Share of citizenships:Q", axis=alt.Axis(format=".1%")),
        color=alt.Color(
            "Region of origin:N",
            scale=alt.Scale(domain=custom_val_order, range=my_custom_palette),
        ),
        order=alt.Order("CustomOrder:Q"),
        tooltip=["Year:O", "Region of origin:N", alt.Tooltip("Share of citizenships:Q", format=".1%")],
    )
)
chart.save(".charts/figure_10_en.html")
chart

alt.Chart(...)

In [38]:
print("DONE.")

DONE.
